In [23]:
import torch
from model import PoseFlowModel
from train import Trainer
from utils.dataset_gen import generate_dataset
from viewer.viewer import Viewer
from viewer.amc_parser import parse_asf
from utils.parser import parse_motion_file
from utils.preprocessing import downsample_motion, tensor_to_motion_frames, motion_frames_to_tensor
from data import metadata, default_skeleton
from utils.motion import Motion, MotionFrame
from transforms3d.euler import euler2mat, mat2euler
from copy import deepcopy
from rotation_conversions import euler_angles_to_matrix
from torch.utils.data import TensorDataset, DataLoader

In [38]:
# x -> (B, L, 3)
a = torch.randn(10, 30, 93)

batch = a.shape[:-1]
a = a.view(-1, 3)


c = []
for b in a:
    c.append(euler_angles_to_matrix(b, 'XYZ').view(-1))

c = torch.stack(c, dim=0)
c = c.reshape(*batch, -1)
a = a.reshape(*batch, -1)

a.shape, c.shape


(torch.Size([10, 30, 93]), torch.Size([10, 30, 279]))

In [9]:
global_rotation = torch.tensor(euler2mat(*torch.tensor([180, -30, -90]).deg2rad().tolist())).to(torch.float32)
v = torch.tensor([1, -4.48948e-011, -5.89033e-027]).to(torch.float32)
global_rotation.inverse() @ v

tensor([-3.7816e-08, -1.0000e+00,  6.5590e-08])

In [ ]:
for bone in default_skeleton.bone_map.values():
    if bone.name == 'root':
        continue
    global_rotation = torch.tensor(euler2mat(*torch.tensor(bone.axis).deg2rad().tolist())).to(torch.float32)
    v = torch.tensor(bone.direction).to(torch.float32)
    d = global_rotation.inverse() @ v
    print(bone.name, d)

In [ ]:
batch_size = 64
block_size = 120
frame_rate = 60
dataset_path = "dataset.bin"
checkpoint_path = "checkpoint.pth"
device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [ ]:
dataset = generate_dataset('data/subjects/05', block_size, frame_rate, window_size=0).to(torch.float32)

min = dataset[:, :, :3].min()
max = dataset[:, :, :3].max()

dataset[:, :, :3] = 2 * ((dataset[:, :, :3] - min) / (max - min)) - 1
data = {
    'dataset': dataset,
    'min': min,
    'max':max 
}
torch.save(data, dataset_path)

In [ ]:
dataset.shape

In [ ]:
model = PoseFlowModel(block_size=block_size, pose_embd=216, block_embd=176, num_layers=12, num_heads=8, dropout=0.2, device=device).to(device)
model.parameter_count()

In [ ]:
trainer = Trainer(model, dataset_path, checkpoint_path, device=device, early_stopper_patience=30, lr=0.0005)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate_loss(trainer.val_loader)

In [ ]:
skeleton = parse_asf('data/subjects/02/02.asf')
motion = parse_motion_file('data/subjects/02/02_02.amc')
motion.frame_rate = metadata['5']['3'].frame_rate
motion = downsample_motion(motion, frame_rate)
motion_tensor = motion_frames_to_tensor(motion.frames)
rotated_motion = rotate(motion)

In [ ]:
# start_motion = motion_tensor[50: 50 + block_size].clone().detach().to(device).to(torch.float32)

start_motion = trainer.test_loader.dataset[10][0].to(device)

model.eval()
generated_motion = []
with torch.no_grad():
    for (pose, _) in zip(model.stream(start_motion.unsqueeze(0).to(device)), range(block_size )):
        generated_motion.append(pose)
generated_motion = torch.cat(generated_motion)

In [ ]:
def scaleup(motion, min, max):
    return motion
    motion[:, :3] = (motion[:, :3] + 1) / 2
    motion[:, :3] = motion[:, :3] * (max - min) + min
    return motion

In [ ]:

real_generated_frames = tensor_to_motion_frames(scaleup(torch.cat([start_motion.cpu(), generated_motion.cpu()], dim=0), trainer.data['min'], trainer.data['max']))

In [ ]:
view = Viewer(skeleton,  rotated_motion.frames)   
view.run()

In [ ]:
def rotate(motion: Motion) -> Motion:
    rotated_motion = Motion(motion.frame_rate)
    rotated_motion.frames = [rotate_motion_frame(frame, torch.tensor(90).deg2rad().item()) for frame in motion.frames]
    return rotated_motion


def rotate_motion_frame(motion_frame: MotionFrame, yaw: float) -> MotionFrame:
    '''
    Rotates a motion frame around the y axis
    '''
    rotated_motion_frame = deepcopy(motion_frame)
    root = rotated_motion_frame['root']
    position = torch.tensor(root[:3]).to(torch.float32)
    rotation = torch.tensor(euler2mat(*torch.tensor(root[3:]).deg2rad().tolist())).to(torch.float32)

    rotation_mat = torch.tensor(euler2mat(0, yaw, 0)).to(torch.float32)
    position = rotation_mat @ position
    rotation = rotation_mat @ rotation

    rotated_motion_frame['root'] = position.tolist() + torch.tensor(mat2euler(rotation.numpy())).rad2deg().tolist()
    return rotated_motion_frame